In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os, json
from glob import glob
import time
import re

%matplotlib inline

In [ ]:
!pip install pattern
!pip install wordfreq
!pip install simplemma

In [6]:
import simplemma
from wordfreq import word_frequency
from pattern.web import plaintext

In [8]:
!mkdir /content/hackfinal
%cd /content/hackfinal

/content/hackfinal


In [9]:
import json
# with open("/content/hackfinal/apps_info.json", 'r') as f:
with open("/content/drive/MyDrive/colab/hackfinal/apps_info.json", 'r') as f:
    info = json.load(f)

In [10]:
regex_tokenize_ru = re.compile("[^а-я]")
regex_tokenize_en = re.compile("[^a-z]")
# regex_tokenize = re.compile("[^\w]")
langdata_en = simplemma.load_data('en')
langdata_ru = simplemma.load_data('ru')

def tokenize_ru(descr_raw):
    descr_plaintext = plaintext(descr_raw).lower()
    words = regex_tokenize_ru.sub(" ", descr_plaintext).split()
    words_lem = list(map(lambda w: simplemma.lemmatize(w, langdata_ru), words))
    return words_lem

def tokenize_en(descr_raw):
    descr_plaintext = plaintext(descr_raw).lower()
    words = regex_tokenize_en.sub(" ", descr_plaintext).split()
    words_lem = list(map(lambda w: simplemma.lemmatize(w, langdata_en), words))
    return words_lem

In [11]:
from tqdm import tqdm
from collections import Counter
keys_list = sorted(list(info.keys()))
words_counter_ru = Counter()
words_counter_en = Counter()
sentences_ru = []
sentences_en = []
skipped = 0
min_sentence_len = 2
for bundle in tqdm(keys_list):
    app_info = info[bundle]
    descr = app_info['description']
    descr = descr.replace('[', ' ')
    descr_words_ru = tokenize_ru(descr)
    descr_words_en = tokenize_en(descr)
    if len(descr_words_ru) > len(descr_words_en):
        if len(descr_words_ru) >= min_sentence_len:
            sentences_ru.append([bundle, descr_words_ru])
        else:
            skipped += 1
        words_counter_ru.update(descr_words_ru)
    else:
        if len(descr_words_en) >= min_sentence_len:
            sentences_en.append([bundle, descr_words_en])
        else:
            skipped += 1
        words_counter_en.update(descr_words_en)
print(len(words_counter_ru), len(words_counter_en))
print("skipped:", skipped)

words_counter_ru_view = list(words_counter_ru.items())
words_counter_ru_view.sort(key=lambda x: (x[1], x[0]), reverse=True)
words_counter_en_view = list(words_counter_en.items())
words_counter_en_view.sort(key=lambda x: (x[1], x[0]), reverse=True)

100%|██████████| 70392/70392 [01:55<00:00, 607.22it/s]


89282 88218
skipped: 378


In [12]:
with open("/content/hackfinal/sentences_ru.txt", "w") as f:
    for (bundle, sent) in sentences_ru:
        f.write(bundle + " " + " ".join(sent)+"\n")

with open("/content/hackfinal/sentences_en.txt", "w") as f:
    for (bundle, sent) in sentences_en:
        f.write(bundle + " " + " ".join(sent)+"\n")

In [13]:
with open("/content/hackfinal/words_ru.txt", "w") as f:
    for (k,v) in words_counter_ru_view:
        f.write(k+'\n')

with open("/content/hackfinal/words_en.txt", "w") as f:
    for (k,v) in words_counter_en_view:
        f.write(k+'\n')

In [14]:
with open("/content/hackfinal/freqs_ru.txt", "w") as f:
    for (word, _) in words_counter_ru_view:
        freq = word_frequency(word, 'ru')
        f.write(word + " " + str(freq) + "\n")

with open("/content/hackfinal/freqs_en.txt", "w") as f:
    for (word, _) in words_counter_en_view:
        freq = word_frequency(word, 'en')
        f.write(word + " " + str(freq) + "\n")

In [15]:
%cd /content/hackfinal

/content/hackfinal


In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!sudo pip2 install .
!make

In [17]:
!cp /content/drive/MyDrive/colab/hackfinal/cc.en.300.bin /content/hackfinal/fastText/
!cp /content/drive/MyDrive/colab/hackfinal/cc.ru.300.bin /content/hackfinal/fastText/

In [18]:
%cd fastText
!python2 download_model.py ru
!python2 download_model.py en

[Errno 2] No such file or directory: 'fastText'
/content/hackfinal/fastText
File exists. Use --overwrite to download anyway.
File exists. Use --overwrite to download anyway.


In [19]:
%cd /content/hackfinal/fastText
t1 = time.time()
!./fasttext print-word-vectors cc.ru.300.bin < /content/hackfinal/words_ru.txt > /content/hackfinal/words_ru_vectors.txt
t2 = time.time()
print(t2-t1)

/content/hackfinal/fastText
tcmalloc: large alloc 4800004096 bytes == 0x55f442a70000 @  0x7f4f25d4d887 0x55f42e6987d3 0x55f42e6a7199 0x55f42e6a76dd 0x55f42e6b27b6 0x55f42e66ca9b 0x7f4f24deabf7 0x55f42e66cb1a
tcmalloc: large alloc 2400002048 bytes == 0x55f560c14000 @  0x7f4f25d4d887 0x55f42e6987d3 0x55f42e6a71e8 0x55f42e6a76dd 0x55f42e6b27b6 0x55f42e66ca9b 0x7f4f24deabf7 0x55f42e66cb1a
123.30860137939453


In [20]:
%cd /content/hackfinal/fastText
t1 = time.time()
!./fasttext print-word-vectors cc.en.300.bin < /content/hackfinal/words_en.txt > /content/hackfinal/words_en_vectors.txt
t2 = time.time()
print(t2-t1)

/content/hackfinal/fastText
tcmalloc: large alloc 4800004096 bytes == 0x55ef1bba4000 @  0x7fbc296d1887 0x55ef0835b7d3 0x55ef0836a199 0x55ef0836a6dd 0x55ef083757b6 0x55ef0832fa9b 0x7fbc2876ebf7 0x55ef0832fb1a
tcmalloc: large alloc 2400002048 bytes == 0x55f039d48000 @  0x7fbc296d1887 0x55ef0835b7d3 0x55ef0836a1e8 0x55ef0836a6dd 0x55ef083757b6 0x55ef0832fa9b 0x7fbc2876ebf7 0x55ef0832fb1a
121.10628819465637


In [24]:
%cd /content/hackfinal
!git clone https://github.com/PrincetonML/SIF.git
%cd SIF
!pip2 install -r /content/hackfinal/SIF/requirements.txt

/content/hackfinal
Cloning into 'SIF'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 2.80 MiB | 11.50 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/hackfinal/SIF
     |████████████████████████████████| 133kB 14.3MB/s 
  Created wheel for lasagne: filename=Lasagne-0.1-cp27-none-any.whl size=79275 sha256=194ed0137ee9e6fb89734cd6844138808c15962e20454fd749ce7db8c3e8cbda
  Stored in directory: /root/.cache/pip/wheels/a5/8e/31/b4cae7e5507f8582e77d7f5cf2815be8820ccacfa0519ca60c
Successfully built lasagne


In [ ]:
# write script for our case of input and output format:

In [36]:
%%writefile /content/hackfinal/SIF/src/sif_embedding_my.py
import sys
sys.path.append('../src')
import data_io, params, SIF_embedding

# input
wordfile = sys.argv[1] # word vector file, can be downloaded from GloVe website
weightfile = sys.argv[2] # each line is a word and its frequency
weightpara = 1e-3 # the parameter in the SIF weighting scheme, usually in the range [3e-5, 3e-3]
rmpc = 1 # number of principal components to remove in SIF weighting scheme
sentences_file = sys.argv[3]
bundles = []
sentences = []
with open(sentences_file, "r") as f:
    for line in f:
        spl = line.rstrip().split(' ')
        bundle = spl[0]
        sentence = ' '.join(spl[1:])
        bundles.append(bundle)
        sentences.append(sentence)

# load word vectors
(words, We) = data_io.getWordmap(wordfile)
# load word weights
word2weight = data_io.getWordWeight(weightfile, weightpara) # word2weight['str'] is the weight for the word 'str'
weight4ind = data_io.getWeight(words, word2weight) # weight4ind[i] is the weight for the i-th word
# load sentences
x, m = data_io.sentences2idx(sentences, words) # x is the array of word indices, m is the binary mask indicating whether there is a word in that location
w = data_io.seq2weight(x, m, weight4ind) # get word weights

# set parameters
params = params.params()
params.rmpc = rmpc
# get SIF embedding
embedding = SIF_embedding.SIF_embedding(We, x, w, params) # embedding[i,:] is the embedding for sentence i

for (bundle, emb) in zip(bundles, embedding):
    print bundle + " " + " ".join(list(map(str, emb)))

Overwriting /content/hackfinal/SIF/src/sif_embedding_my.py


In [ ]:
# fix one bug in original repo source code

In [37]:
%%writefile /content/hackfinal/SIF/src/data_io.py
from __future__ import print_function

import numpy as np
import pickle
from tree import tree
#from theano import config

def getWordmap(textfile):
    words={}
    We = []
    f = open(textfile,'r')
    lines = f.readlines()
    for (n,i) in enumerate(lines):
        i=i.split()
        j = 1
        v = []
        while j < len(i):
            v.append(float(i[j]))
            j += 1
        words[i[0]]=n
        We.append(v)
    return (words, np.array(We))

def prepare_data(list_of_seqs):
    lengths = [len(s) for s in list_of_seqs]
    n_samples = len(list_of_seqs)
    maxlen = np.max(lengths)
    x = np.zeros((n_samples, maxlen)).astype('int32')
    x_mask = np.zeros((n_samples, maxlen)).astype('float32')
    for idx, s in enumerate(list_of_seqs):
        x[idx, :lengths[idx]] = s
        x_mask[idx, :lengths[idx]] = 1.
    x_mask = np.asarray(x_mask, dtype='float32')
    return x, x_mask

def lookupIDX(words,w):
    w = w.lower()
    if len(w) > 1 and w[0] == '#':
        w = w.replace("#","")
    if w in words:
        return words[w]
    elif 'UUUNKKK' in words:
        return words['UUUNKKK']
    else:
        return len(words) - 1

def getSeq(p1,words):
    p1 = p1.split()
    X1 = []
    for i in p1:
        X1.append(lookupIDX(words,i))
    return X1

def getSeqs(p1,p2,words):
    p1 = p1.split()
    p2 = p2.split()
    X1 = []
    X2 = []
    for i in p1:
        X1.append(lookupIDX(words,i))
    for i in p2:
        X2.append(lookupIDX(words,i))
    return X1, X2

def get_minibatches_idx(n, minibatch_size, shuffle=False):
    idx_list = np.arange(n, dtype="int32")

    if shuffle:
        np.random.shuffle(idx_list)

    minibatches = []
    minibatch_start = 0
    for i in range(n // minibatch_size):
        minibatches.append(idx_list[minibatch_start:
        minibatch_start + minibatch_size])
        minibatch_start += minibatch_size

    if (minibatch_start != n):
        minibatches.append(idx_list[minibatch_start:])

    return zip(range(len(minibatches)), minibatches)

def getSimEntDataset(f,words,task):
    data = open(f,'r')
    lines = data.readlines()
    examples = []
    for i in lines:
        i=i.strip()
        if(len(i) > 0):
            i=i.split('\t')
            if len(i) == 3:
                if task == "sim":
                    e = (tree(i[0], words), tree(i[1], words), float(i[2]))
                    examples.append(e)
                elif task == "ent":
                    e = (tree(i[0], words), tree(i[1], words), i[2])
                    examples.append(e)
                else:
                    raise ValueError('Params.traintype not set correctly.')

            else:
                print(i)
    return examples

def getSentimentDataset(f,words):
    data = open(f,'r')
    lines = data.readlines()
    examples = []
    for i in lines:
        i=i.strip()
        if(len(i) > 0):
            i=i.split('\t')
            if len(i) == 2:
                e = (tree(i[0], words), i[1])
                examples.append(e)
            else:
                print(i)
    return examples

def getDataSim(batch, nout):
    g1 = []
    g2 = []
    for i in batch:
        g1.append(i[0].embeddings)
        g2.append(i[1].embeddings)

    g1x, g1mask = prepare_data(g1)
    g2x, g2mask = prepare_data(g2)

    scores = []
    if nout <=0:
        return (scores, g1x, g1mask, g2x, g2mask)

    for i in batch:
        temp = np.zeros(nout)
        score = float(i[2])
        ceil, fl = int(np.ceil(score)), int(np.floor(score))
        if ceil == fl:
            temp[fl - 1] = 1
        else:
            temp[fl - 1] = ceil - score
            temp[ceil - 1] = score - fl
        scores.append(temp)
    scores = np.matrix(scores) + 0.000001
    scores = np.asarray(scores, dtype='float32')
    return (scores, g1x, g1mask, g2x, g2mask)

def getDataEntailment(batch):
    g1 = []; g2 = []
    for i in batch:
        g1.append(i[0].embeddings)
        g2.append(i[1].embeddings)

    g1x, g1mask = prepare_data(g1)
    g2x, g2mask = prepare_data(g2)

    scores = []
    for i in batch:
        temp = np.zeros(3)
        label = i[2].strip()
        if label == "CONTRADICTION":
            temp[0]=1
        if label == "NEUTRAL":
            temp[1]=1
        if label == "ENTAILMENT":
            temp[2]=1
        scores.append(temp)
    scores = np.matrix(scores)+0.000001
    scores = np.asarray(scores,dtype='float32')
    return (scores,g1x,g1mask,g2x,g2mask)

def getDataSentiment(batch):
    g1 = []
    for i in batch:
        g1.append(i[0].embeddings)

    g1x, g1mask = prepare_data(g1)

    scores = []
    for i in batch:
        temp = np.zeros(2)
        label = i[1].strip()
        if label == "0":
            temp[0]=1
        if label == "1":
            temp[1]=1
        scores.append(temp)
    scores = np.matrix(scores)+0.000001
    scores = np.asarray(scores,dtype='float32')
    return (scores,g1x,g1mask)

def sentences2idx(sentences, words):
    """
    Given a list of sentences, output array of word indices that can be fed into the algorithms.
    :param sentences: a list of sentences
    :param words: a dictionary, words['str'] is the indices of the word 'str'
    :return: x1, m1. x1[i, :] is the word indices in sentence i, m1[i,:] is the mask for sentence i (0 means no word at the location)
    """
    seq1 = []
    for i in sentences:
        seq1.append(getSeq(i,words))
    x1,m1 = prepare_data(seq1)
    return x1, m1


def sentiment2idx(sentiment_file, words):
    """
    Read sentiment data file, output array of word indices that can be fed into the algorithms.
    :param sentiment_file: file name
    :param words: a dictionary, words['str'] is the indices of the word 'str'
    :return: x1, m1, golds. x1[i, :] is the word indices in sentence i, m1[i,:] is the mask for sentence i (0 means no word at the location), golds[i] is the label (0 or 1) for sentence i.
    """
    f = open(sentiment_file,'r')
    lines = f.readlines()
    golds = []
    seq1 = []
    for i in lines:
        i = i.split("\t")
        p1 = i[0]; score = int(i[1]) # score are labels 0 and 1
        X1 = getSeq(p1,words)
        seq1.append(X1)
        golds.append(score)
    x1,m1 = prepare_data(seq1)
    return x1, m1, golds

def sim2idx(sim_file, words):
    """
    Read similarity data file, output array of word indices that can be fed into the algorithms.
    :param sim_file: file name
    :param words: a dictionary, words['str'] is the indices of the word 'str'
    :return: x1, m1, x2, m2, golds. x1[i, :] is the word indices in the first sentence in pair i, m1[i,:] is the mask for the first sentence in pair i (0 means no word at the location), golds[i] is the score for pair i (float). x2 and m2 are similar to x1 and m2 but for the second sentence in the pair.
    """
    f = open(sim_file,'r')
    lines = f.readlines()
    golds = []
    seq1 = []
    seq2 = []
    for i in lines:
        i = i.split("\t")
        p1 = i[0]; p2 = i[1]; score = float(i[2])
        X1, X2 = getSeqs(p1,p2,words)
        seq1.append(X1)
        seq2.append(X2)
        golds.append(score)
    x1,m1 = prepare_data(seq1)
    x2,m2 = prepare_data(seq2)
    return x1, m1, x2, m2, golds

def entailment2idx(sim_file, words):
    """
    Read similarity data file, output array of word indices that can be fed into the algorithms.
    :param sim_file: file name
    :param words: a dictionary, words['str'] is the indices of the word 'str'
    :return: x1, m1, x2, m2, golds. x1[i, :] is the word indices in the first sentence in pair i, m1[i,:] is the mask for the first sentence in pair i (0 means no word at the location), golds[i] is the label for pair i (CONTRADICTION NEUTRAL ENTAILMENT). x2 and m2 are similar to x1 and m2 but for the second sentence in the pair.
    """
    f = open(sim_file,'r')
    lines = f.readlines()
    golds = []
    seq1 = []
    seq2 = []
    for i in lines:
        i = i.split("\t")
        p1 = i[0]; p2 = i[1]; score = i[2]
        X1, X2 = getSeqs(p1,p2,words)
        seq1.append(X1)
        seq2.append(X2)
        golds.append(score)
    x1,m1 = prepare_data(seq1)
    x2,m2 = prepare_data(seq2)
    return x1, m1, x2, m2, golds

def getWordWeight(weightfile, a=1e-3):
    if a <=0: # when the parameter makes no sense, use unweighted
        a = 1.0

    word2weight = {}
    with open(weightfile) as f:
        lines = f.readlines()
    N = 0
    for i in lines:
        i=i.strip()
        if(len(i) > 0):
            i=i.split()
            if(len(i) == 2):
                word2weight[i[0]] = float(i[1])
                N += float(i[1])
            else:
                print(i)
    for key, value in word2weight.iteritems():
        word2weight[key] = a / (a + value/N)
    return word2weight

def getWeight(words, word2weight):
    weight4ind = {}
    for word, ind in words.iteritems():
        if word in word2weight:
            weight4ind[ind] = word2weight[word]
        else:
            weight4ind[ind] = 1.0
    return weight4ind

def seq2weight(seq, mask, weight4ind):
    weight = np.zeros(seq.shape).astype('float32')
    for i in xrange(seq.shape[0]):
        for j in xrange(seq.shape[1]):
            if mask[i,j] > 0 and seq[i,j] >= 0:
                weight[i,j] = weight4ind[seq[i,j]]
    weight = np.asarray(weight, dtype='float32')
    return weight

def getIDFWeight(wordfile, save_file=''):
    def getDataFromFile(f, words):
        f = open(f,'r')
        lines = f.readlines()
        golds = []
        seq1 = []
        seq2 = []
        for i in lines:
            i = i.split("\t")
            p1 = i[0]; p2 = i[1]; score = float(i[2])
            X1, X2 = getSeqs(p1,p2,words)
            seq1.append(X1)
            seq2.append(X2)
            golds.append(score)
        x1,m1 = prepare_data(seq1)
        x2,m2 = prepare_data(seq2)
        return x1,m1,x2,m2

    prefix = "../data/"
    farr = ["MSRpar2012"]
    #farr = ["MSRpar2012",
    #        "MSRvid2012",
    #        "OnWN2012",
    #        "SMTeuro2012",
    #        "SMTnews2012", # 4
    #        "FNWN2013",
    #        "OnWN2013",
    #        "SMT2013",
    #        "headline2013", # 8
    #        "OnWN2014",
    #        "deft-forum2014",
    #        "deft-news2014",
    #        "headline2014",
    #        "images2014",
    #        "tweet-news2014", # 14
    #        "answer-forum2015",
    #        "answer-student2015",
    #        "belief2015",
    #        "headline2015",
    #        "images2015",    # 19
    #        "sicktest",
    #        "twitter",
    #        "JHUppdb",
    #        "anno-dev",
    #        "anno-test"]
    (words, We) = getWordmap(wordfile)
    df = np.zeros((len(words),))
    dlen = 0
    for f in farr:
        g1x,g1mask,g2x,g2mask = getDataFromFile(prefix+f, words)
        dlen += g1x.shape[0]
        dlen += g2x.shape[0]
        for i in xrange(g1x.shape[0]):
            for j in xrange(g1x.shape[1]):
                if g1mask[i,j] > 0:
                    df[g1x[i,j]] += 1
        for i in xrange(g2x.shape[0]):
            for j in xrange(g2x.shape[1]):
                if g2mask[i,j] > 0:
                    df[g2x[i,j]] += 1

    weight4ind = {}
    for i in xrange(len(df)):
        weight4ind[i] = np.log2((dlen+2.0)/(1.0+df[i]))
    if save_file:
        pickle.dump(weight4ind, open(save_file, 'w'))
    return weight4ind

Overwriting /content/hackfinal/SIF/src/data_io.py


In [38]:
%cd /content/hackfinal/SIF/src
t1 = time.time()
!python2 sif_embedding_my.py /content/hackfinal/words_en_vectors.txt /content/hackfinal/freqs_en.txt /content/hackfinal/sentences_en.txt > /content/hackfinal/embeddings_en.txt
t2 = time.time()
print(t2-t1)

/content/hackfinal/SIF/src
170.21436738967896


In [40]:
%cd /content/hackfinal/SIF/src
t1 = time.time()
!python2 sif_embedding_my.py /content/hackfinal/words_ru_vectors.txt /content/hackfinal/freqs_ru.txt /content/hackfinal/sentences_ru.txt > /content/hackfinal/embeddings_ru.txt
t2 = time.time()
print(t2-t1)

/content/hackfinal/SIF/src
112.51141357421875


In [ ]:
# convert obtained embeddings_*.txt to more compact format (2 embedding_*.npy files and 2 bundles_*.txt files)
bundles_list_en = []
vectors_en = []
with open("/content/hackfinal/embeddings_en.txt", "r") as f:
    for line in f:
        spl = line.split(' ')
        bundle = spl[0]
        vec = np.array(list(map(float,spl[1:])))
        bundles_list_en.append(bundle)
        vectors_en.append(vec)
vectors_en = np.array(vectors_en)

bundles_list_ru = []
vectors_ru = []
with open("/content/hackfinal/embeddings_ru.txt", "r") as f:
    for line in f:
        spl = line.split(' ')
        bundle = spl[0]
        vec = np.array(list(map(float,spl[1:])))
        bundles_list_ru.append(bundle)
        vectors_ru.append(vec)
vectors_ru = np.array(vectors_ru)

with open("/content/hackfinal/bundles_ru.txt", "w") as f:
    for bundle in bundles_list_ru:
        f.write(bundle+'\n')

with open("/content/hackfinal/bundles_en.txt", "w") as f:
    for bundle in bundles_list_en:
        f.write(bundle+'\n')

with open('/content/hackfinal/embeddings_ru.npy', 'wb') as f:
    np.save(f, vectors_ru)

with open('/content/hackfinal/embeddings_en.npy', 'wb') as f:
    np.save(f, vectors_en)